In [1]:
# load data to 
import matplotlib.image as img
import numpy as np
import os

train_dataset = "./handwrite__detect/train_image"
test_dataset = "./handwrite__detect/test_image"

def loadData(src):
    x = np.empty([0, 28, 28, 1])
    y = np.empty([0])
    for root, dirs, files in os.walk(src):
        root_split = root.split("\\")

        if len(root_split) == 3:
            # print(root)
            for f in files:
                in_file = os.path.join(root, f)
                y = np.append(y, [root_split[2]], axis=0)  # index
                # image only black(255, 255, 255) or white(0, 0, 0), so just choose the first number
                np_img = img.imread(in_file)[:,:,0].reshape([1, 28, 28, 1])  
                x = np.append(x, np_img, axis=0)  # image
    return x, y
    
# x: image list, # y: number list
x_train, y_train = loadData(train_dataset)
x_test, y_test = loadData(test_dataset)

print(len(x_train))
print(len(x_test))

2450
1700


In [2]:
# make a CNN model
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam  # SGD

# shape = [-1, 28, 28, 1] <=> [-1, 784]
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential(
    [
        # -- ways one -- acc:0.93
        # Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        # MaxPooling2D((2, 2)),
        # Flatten(),
        # Dense(100, activation='relu'),
        # Dense(10, activation='softmax')
        # -- ways two -- acc:0.95
        Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(10, activation="softmax")
    ]
)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [3]:
# training
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=10,
    validation_data=(x_test, y_test)
)
model.save("chinese_number_identification_model.h5")

Epoch 1/10
20/20 [==============================] - 20s 186ms/step - loss: 1.9690 - accuracy: 0.3207 - val_loss: 0.7730 - val_accuracy: 0.7800
Epoch 2/10
20/20 [==============================] - 3s 146ms/step - loss: 0.6980 - accuracy: 0.7818 - val_loss: 0.4573 - val_accuracy: 0.8565
Epoch 3/10
20/20 [==============================] - 3s 145ms/step - loss: 0.3708 - accuracy: 0.8849 - val_loss: 0.3629 - val_accuracy: 0.8724
Epoch 4/10
20/20 [==============================] - 3s 147ms/step - loss: 0.2455 - accuracy: 0.9215 - val_loss: 0.2912 - val_accuracy: 0.9012
Epoch 5/10
20/20 [==============================] - 3s 147ms/step - loss: 0.1763 - accuracy: 0.9476 - val_loss: 0.2705 - val_accuracy: 0.9059
Epoch 6/10
20/20 [==============================] - 3s 150ms/step - loss: 0.1405 - accuracy: 0.9534 - val_loss: 0.2500 - val_accuracy: 0.9112
Epoch 7/10
20/20 [==============================] - 3s 151ms/step - loss: 0.1315 - accuracy: 0.9572 - val_loss: 0.2190 - val_accuracy: 0.9271
Epoch

In [4]:
# answer viewing
score = model.evaluate(x_test, y_test)

54/54 [==============================] - 1s 11ms/step - loss: 0.2021 - accuracy: 0.9318
